In [10]:
import requests
import csv
import json
from bs4 import BeautifulSoup

white_actress=[]
white_actress_list_name=[]#white actress
white_actress_list_img=[];#white actress img url
white_actress_url={};
white_actress_birth={};
white_actress_age={}; #white actress age (2019)

req = requests.get('https://www.imdb.com/list/ls052159785/')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
names=soup.select(
        '#main > div > div.lister.list.detail.sub-list > div.lister-list > div > div.lister-item-content > h3 > a'
)

for name in names:
    white_actress_dict=dict()
    white_actress_dict['name']=str(name.text).replace("\n","").replace(" ","")
    white_actress_url[str(name.text).replace("\n","").replace(" ","")]='https://www.imdb.com'+name.get('href')
    white_actress_list_name.append(white_actress_dict)

for key in white_actress_url:
    req = requests.get(white_actress_url[key])
    html = req.text
    soup = BeautifulSoup(html, 'html.parser')
    bday = soup.select(
    '#name-born-info'
    )
    for info in bday:
        if(info.select('h4')):
            if(info.select('time')):
                for date in info.select('time'):
                    white_actress_birth[key]=date['datetime']
                    white_actress_age[key]=2019-(int(str(date['datetime']).split('-')[0]))

req = requests.get('https://www.imdb.com/list/ls052159785/')
html = req.text
soup = BeautifulSoup(html, 'html.parser')
img=soup.select(
    '#main > div > div.lister.list.detail.sub-list > div.lister-list > div > div.lister-item-image > a > img'
)

for src in img:
    no_image_url="https://m.media-amazon.com/images/G/01/imdb/images/nopicture/medium/name-2135195744._CB470041852_.png"
    if(src.get('src')!=no_image_url):
        white_actress_dict=dict()
        white_actress_dict['src']=str(src.get('src') )
        white_actress_list_img.append(white_actress_dict)
    else: #when there is no image
        white_actress_dict=dict()
        white_actress_dict['src']="" 
        white_actress_list_img.append(white_actress_dict)
        
with open('white_actress_name.json', 'w+') as json_file:
    json.dump(white_actress_list_name, json_file)
    
with open('white_actress_img.json', 'w+') as json_file:
    json.dump(white_actress_list_img, json_file)        

i=0
while i<100:
    list1=[]
    list1.append(white_actress_list_name[i]['name'])
    list1.append(white_actress_list_img[i]['src'])
    for key in white_actress_birth:
        if(str(key) == white_actress_list_name[i]['name']):
            list1.append(str(white_actress_birth[key]))
            for k in white_actress_age:
                if(str(k) == white_actress_list_name[i]['name']):
                    list1.append(str(white_actress_age[k]))
    white_actress.append(list1)
    i=i+1
    
for index in white_actress:
    if(len(index)!=4):
        white_actress.remove(index)
        
with open('white_actress.csv', 'w', encoding='utf-8') as csv_file:
    writer = csv.writer(csv_file, dialect='excel')
    writer.writerows(white_actress)
